🧩 Parte 1 – Imports, tabela de estados, anos, sessão

In [0]:

import pandas as pd
import numpy as np
import requests

# 1. Tabela de estados
estados = pd.DataFrame({
    'sigla': ["AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA",
              "MG", "MS", "MT", "PA", "PB", "PE", "PI", "PR", "RJ", "RN",
              "RO", "RR", "RS", "SC", "SE", "SP", "TO"],
    'codigo': ["12", "27", "16", "13", "29", "23", "53", "32", "52", "21",
               "31", "50", "51", "15", "25", "26", "22", "41", "33", "24",
               "11", "14", "43", "42", "28", "35", "17"]
})

# 2. Intervalo de anos 
exercicio = range(2013, 2025)  # 2013 até 2024

# 3. Lista que vai guardar os DataFrames retornados pela API
resultados_lista = []

# 4. URL base da API SICONFI (DCA)
url_base = "https://apidatalake.tesouro.gov.br/ords/siconfi/tt/dca"

# 5. Session para reaproveitar conexão HTTP
session = requests.Session()


🧩 Parte 2 – Loop de coleta na API

In [0]:
for uf_codigo in estados['codigo']:
    sigla_uf = estados.loc[estados['codigo'] == uf_codigo, 'sigla'].iloc[0]

    for ano in exercicio:
        params = {
            'an_exercicio': ano,
            'id_ente': uf_codigo
        }

        try:
            response = session.get(url_base, params=params, timeout=30)
            status = response.status_code

            print(f"Status da resposta: {status} para UF {sigla_uf} ano {ano}")

            if status == 200:
                dados = response.json()
                items_list = dados.get('items', [])

                if not items_list:
                    print(f"  -> Aviso: Nenhum item retornado para {sigla_uf} em {ano}.")
                    continue

                items = pd.DataFrame(items_list)

                # Adiciona metadados
                # items['UF'] = sigla_uf 
                items['Codigo_UF'] = uf_codigo
                items['Ano'] = ano

                # Se existir coluna 'valor', converte pra numérico
                if 'valor' in items.columns:
                    items['valor'] = pd.to_numeric(items['valor'], errors='coerce')

                resultados_lista.append(items)

            else:
                print(f"  -> Erro na requisição para {sigla_uf} em {ano}. Status: {status}")
                continue

        except requests.exceptions.RequestException as e:
            print(f"  -> Erro de conexão para {sigla_uf} em {ano}: {e}")
            continue


🧩 Parte 3 – Juntar tudo e salvar como BRONZE no Databricks

In [0]:
# 1) Garante que existem dados
if len(resultados_lista) == 0:
    raise ValueError("Nenhum dado foi retornado da API. Verifique conexão ou parâmetros.")

# 2) Junta tudo num único DataFrame pandas
resultados_brutos = pd.concat(resultados_lista, ignore_index=True)

print("Quantidade de linhas coletadas:", len(resultados_brutos))
print("Colunas disponíveis:", list(resultados_brutos.columns))

# 3) Converte para Spark DataFrame
spark_df_bronze = spark.createDataFrame(resultados_brutos)

# 4) Define explicitamente o catálogo onde salvar as tabelas
target_catalog = "workspace"  

print("Catálogo escolhido para salvar as tabelas:", target_catalog)

# 5) Usa o catálogo 'workspace'
spark.sql(f"USE CATALOG {target_catalog}")

# 6) Cria o SCHEMA mvp dentro desse catálogo, se não existir
spark.sql("CREATE SCHEMA IF NOT EXISTS mvp")

# 7) Salva a tabela BRONZE no catálogo/esquema escolhidos
tabela_bronze = f"{target_catalog}.mvp.bronze_dca"

spark_df_bronze.write.format("delta").mode("overwrite").saveAsTable(tabela_bronze)

print(f"✅ Tabela BRONZE criada com sucesso em: {tabela_bronze}")


🧩 Parte 4 – Conferir a tabela BRONZE

In [0]:
df_bronze = spark.table("workspace.mvp.bronze_dca")

display(df_bronze.limit(20))
df_bronze.printSchema()
